# Python 后端认证与安全基础

## 学习目标
- 掌握现代 Web 应用的主要认证方式
- 理解密码安全和加密解密基础知识
- 学会实现安全的用户认证系统
- 了解常见的 Web 安全威胁及防护措施
- 掌握 JWT、OAuth 等现代认证技术
- 学会使用 Python 加密库进行安全编程

## 为什么安全很重要？
在现代 Web 开发中，**安全不是可选项，而是必需品**。数据泄露、身份盗用、恶意攻击等安全事件层出不穷，作为后端开发者，我们必须：

- **保护用户数据**：防止敏感信息泄露
- **验证用户身份**：确保只有授权用户能访问资源  
- **防范恶意攻击**：抵御各种网络攻击
- **遵守法律法规**：满足 GDPR、等保等合规要求

> 💡 **PyBackendPro 提醒**：安全是一个系统工程，需要在设计阶段就考虑，而不是事后补救。


In [ ]:
import hashlib
import hmac
import secrets
import base64
import json
from typing import Dict, Any, Optional, Tuple
from datetime import datetime, timedelta
import logging
from dataclasses import dataclass
from cryptography.fernet import Fernet
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
import bcrypt
import jwt
import os
from enum import Enum

# 配置日志
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

print("🔐 Python 认证与安全基础教程")
print("=" * 50)
print("导入的安全相关库：")
print("✅ hashlib - 哈希算法")
print("✅ hmac - 消息认证码")  
print("✅ secrets - 安全随机数")
print("✅ cryptography - 现代加密库")
print("✅ bcrypt - 密码哈希专用")
print("✅ jwt - JSON Web Token")
print("=" * 50)


## 1. 密码安全基础

### 密码安全的核心原则
1. **永远不要明文存储密码** - 这是最基本的原则
2. **使用强密码哈希算法** - 如 bcrypt、scrypt、Argon2
3. **添加盐值 (Salt)** - 防止彩虹表攻击
4. **慢哈希算法** - 增加暴力破解成本
5. **密码复杂度要求** - 长度、字符类型要求

### 常见的密码安全威胁
- **彩虹表攻击**：预计算的哈希-密码对照表
- **暴力破解**：尝试所有可能的密码组合
- **字典攻击**：使用常见密码列表
- **社会工程学**：通过非技术手段获取密码


In [ ]:
class PasswordSecurity:
    """
    密码安全处理类 - 生产环境标准实现
    """
    
    def __init__(self):
        self.logger = logging.getLogger(self.__class__.__name__)
    
    def generate_salt(self, length: int = 32) -> str:
        """
        生成安全的随机盐值
        
        Args:
            length: 盐值长度
            
        Returns:
            Base64 编码的盐值
        """
        salt = secrets.token_bytes(length)
        return base64.b64encode(salt).decode('utf-8')
    
    def hash_password_pbkdf2(self, password: str, salt: str = None) -> Tuple[str, str]:
        """
        使用 PBKDF2 哈希密码（推荐用于理解原理）
        
        Args:
            password: 明文密码
            salt: 盐值，如果为 None 则自动生成
            
        Returns:
            (哈希值, 盐值) 元组
        """
        if salt is None:
            salt = self.generate_salt()
        
        # 将盐值转换为字节
        salt_bytes = base64.b64decode(salt.encode('utf-8'))
        
        # 配置 PBKDF2 参数
        kdf = PBKDF2HMAC(
            algorithm=hashes.SHA256(),
            length=32,  # 输出长度
            salt=salt_bytes,
            iterations=100000,  # 迭代次数，可根据性能调整
        )
        
        # 计算哈希
        key = kdf.derive(password.encode('utf-8'))
        hash_value = base64.b64encode(key).decode('utf-8')
        
        self.logger.info(f"密码哈希完成，使用 PBKDF2-SHA256，迭代 100000 次")
        return hash_value, salt
    
    def verify_password_pbkdf2(self, password: str, hash_value: str, salt: str) -> bool:
        """
        验证 PBKDF2 哈希的密码
        
        Args:
            password: 待验证的明文密码
            hash_value: 存储的哈希值
            salt: 盐值
            
        Returns:
            验证结果
        """
        try:
            computed_hash, _ = self.hash_password_pbkdf2(password, salt)
            return hmac.compare_digest(hash_value, computed_hash)
        except Exception as e:
            self.logger.error(f"密码验证失败: {e}")
            return False
    
    def hash_password_bcrypt(self, password: str) -> str:
        """
        使用 bcrypt 哈希密码（生产环境推荐）
        
        Args:
            password: 明文密码
            
        Returns:
            bcrypt 哈希值（包含盐值）
        """
        # bcrypt 自动处理盐值生成和存储
        salt = bcrypt.gensalt(rounds=12)  # rounds 越高越安全但越慢
        hashed = bcrypt.hashpw(password.encode('utf-8'), salt)
        
        self.logger.info(f"密码哈希完成，使用 bcrypt，成本因子 12")
        return hashed.decode('utf-8')
    
    def verify_password_bcrypt(self, password: str, hashed: str) -> bool:
        """
        验证 bcrypt 哈希的密码
        
        Args:
            password: 待验证的明文密码
            hashed: bcrypt 哈希值
            
        Returns:
            验证结果
        """
        try:
            return bcrypt.checkpw(password.encode('utf-8'), hashed.encode('utf-8'))
        except Exception as e:
            self.logger.error(f"bcrypt 密码验证失败: {e}")
            return False
    
    def validate_password_strength(self, password: str) -> Dict[str, Any]:
        """
        验证密码强度
        
        Args:
            password: 待验证的密码
            
        Returns:
            验证结果字典
        """
        result = {
            'is_valid': True,
            'score': 0,
            'issues': [],
            'suggestions': []
        }
        
        # 长度检查
        if len(password) < 8:
            result['issues'].append('密码长度至少需要 8 位')
            result['is_valid'] = False
        elif len(password) >= 12:
            result['score'] += 2
        else:
            result['score'] += 1
        
        # 字符类型检查
        has_lower = any(c.islower() for c in password)
        has_upper = any(c.isupper() for c in password)
        has_digit = any(c.isdigit() for c in password)
        has_special = any(c in '!@#$%^&*()_+-=[]{}|;:,.<>?' for c in password)
        
        char_types = sum([has_lower, has_upper, has_digit, has_special])
        
        if char_types < 3:
            result['issues'].append('密码应包含至少 3 种字符类型（大写、小写、数字、特殊字符）')
            result['is_valid'] = False
        
        result['score'] += char_types
        
        # 常见密码检查（简化版）
        common_passwords = ['123456', 'password', '123456789', 'qwerty', 'abc123']
        if password.lower() in common_passwords:
            result['issues'].append('密码过于常见，容易被破解')
            result['is_valid'] = False
            result['score'] = 0
        
        # 生成建议
        if not has_lower:
            result['suggestions'].append('添加小写字母')
        if not has_upper:
            result['suggestions'].append('添加大写字母')
        if not has_digit:
            result['suggestions'].append('添加数字')
        if not has_special:
            result['suggestions'].append('添加特殊字符')
        
        return result

# 密码安全演示
print("=== 密码安全演示 ===")
pwd_security = PasswordSecurity()

# 测试密码
test_password = "MySecurePassword123!"
weak_password = "123456"

print(f"\n1. 密码强度验证:")
strong_result = pwd_security.validate_password_strength(test_password)
weak_result = pwd_security.validate_password_strength(weak_password)

print(f"强密码 '{test_password}' 验证结果:")
print(f"  有效: {strong_result['is_valid']}")
print(f"  评分: {strong_result['score']}/6")
print(f"  问题: {strong_result['issues']}")

print(f"\n弱密码 '{weak_password}' 验证结果:")
print(f"  有效: {weak_result['is_valid']}")
print(f"  评分: {weak_result['score']}/6")
print(f"  问题: {weak_result['issues']}")
print(f"  建议: {weak_result['suggestions']}")


In [ ]:
# 密码哈希实战演示

print("\n2. 密码哈希对比演示:")

# bcrypt 哈希（推荐）
bcrypt_hash = pwd_security.hash_password_bcrypt(test_password)
print(f"bcrypt 哈希: {bcrypt_hash}")
print(f"bcrypt 验证: {pwd_security.verify_password_bcrypt(test_password, bcrypt_hash)}")
print(f"bcrypt 错误密码验证: {pwd_security.verify_password_bcrypt('wrong_password', bcrypt_hash)}")

# PBKDF2 哈希（教学用）
pbkdf2_hash, salt = pwd_security.hash_password_pbkdf2(test_password)
print(f"\nPBKDF2 哈希: {pbkdf2_hash}")
print(f"PBKDF2 盐值: {salt}")
print(f"PBKDF2 验证: {pwd_security.verify_password_pbkdf2(test_password, pbkdf2_hash, salt)}")
print(f"PBKDF2 错误密码验证: {pwd_security.verify_password_pbkdf2('wrong_password', pbkdf2_hash, salt)}")

# 演示为什么不能使用简单哈希
print("\n3. 为什么不能使用简单哈希:")
simple_hash = hashlib.sha256(test_password.encode()).hexdigest()
print(f"简单 SHA256 哈希: {simple_hash}")
print("⚠️  问题: 相同密码总是产生相同哈希，容易受到彩虹表攻击")

# 相同密码的不同 bcrypt 哈希
hash1 = pwd_security.hash_password_bcrypt(test_password)
hash2 = pwd_security.hash_password_bcrypt(test_password)
print(f"\n相同密码的不同 bcrypt 哈希:")
print(f"哈希1: {hash1}")
print(f"哈希2: {hash2}")
print(f"两个哈希相同吗? {hash1 == hash2}")
print("✅ bcrypt 每次都生成不同的哈希，即使密码相同（因为盐值不同）")


## 2. JWT (JSON Web Token) 认证

### JWT 的特点
- **无状态**：服务器不需要存储会话信息
- **可扩展**：适合分布式系统
- **自包含**：包含用户信息，减少数据库查询
- **标准化**：RFC 7519 标准

### JWT 结构
JWT 由三部分组成，用点（.）分隔：
1. **Header**：算法和令牌类型
2. **Payload**：声明（用户信息）
3. **Signature**：签名（验证完整性）

格式：`header.payload.signature`

### 使用场景
- **API 认证**：RESTful API 的标准认证方式
- **单点登录**：跨域认证
- **微服务**：服务间认证
- **移动应用**：无需维护会话状态


In [ ]:
class JWTManager:
    """
    JWT 管理器 - 生产环境实现
    """
    
    def __init__(self, secret_key: str = None, algorithm: str = 'HS256'):
        self.secret_key = secret_key or secrets.token_urlsafe(32)
        self.algorithm = algorithm
        self.logger = logging.getLogger(self.__class__.__name__)
        
        # 令牌过期时间配置
        self.access_token_expire_minutes = 15  # 访问令牌15分钟过期
        self.refresh_token_expire_days = 7     # 刷新令牌7天过期
    
    def create_access_token(self, user_data: Dict[str, Any]) -> str:
        """
        创建访问令牌
        
        Args:
            user_data: 用户数据字典
            
        Returns:
            JWT 访问令牌
        """
        now = datetime.utcnow()
        expire = now + timedelta(minutes=self.access_token_expire_minutes)
        
        payload = {
            'user_id': user_data.get('user_id'),
            'username': user_data.get('username'),
            'email': user_data.get('email'),
            'roles': user_data.get('roles', []),
            'iat': now,  # issued at
            'exp': expire,  # expiration time
            'type': 'access'
        }
        
        token = jwt.encode(payload, self.secret_key, algorithm=self.algorithm)
        self.logger.info(f"创建访问令牌成功，用户: {user_data.get('username')}")
        return token
    
    def create_refresh_token(self, user_id: int) -> str:
        """
        创建刷新令牌
        
        Args:
            user_id: 用户ID
            
        Returns:
            JWT 刷新令牌
        """
        now = datetime.utcnow()
        expire = now + timedelta(days=self.refresh_token_expire_days)
        
        payload = {
            'user_id': user_id,
            'iat': now,
            'exp': expire,
            'type': 'refresh'
        }
        
        token = jwt.encode(payload, self.secret_key, algorithm=self.algorithm)
        self.logger.info(f"创建刷新令牌成功，用户ID: {user_id}")
        return token
    
    def verify_token(self, token: str) -> Optional[Dict[str, Any]]:
        """
        验证令牌
        
        Args:
            token: JWT 令牌
            
        Returns:
            解码后的 payload 或 None
        """
        try:
            payload = jwt.decode(token, self.secret_key, algorithms=[self.algorithm])
            
            # 检查令牌类型
            token_type = payload.get('type', 'access')
            self.logger.info(f"令牌验证成功，类型: {token_type}")
            
            return payload
            
        except jwt.ExpiredSignatureError:
            self.logger.warning("令牌已过期")
            return None
        except jwt.InvalidTokenError as e:
            self.logger.error(f"无效令牌: {e}")
            return None
    
    def refresh_access_token(self, refresh_token: str, user_data: Dict[str, Any]) -> Optional[str]:
        """
        使用刷新令牌生成新的访问令牌
        
        Args:
            refresh_token: 刷新令牌
            user_data: 用户数据
            
        Returns:
            新的访问令牌或 None
        """
        payload = self.verify_token(refresh_token)
        
        if not payload or payload.get('type') != 'refresh':
            self.logger.error("无效的刷新令牌")
            return None
        
        # 验证用户ID匹配
        if payload.get('user_id') != user_data.get('user_id'):
            self.logger.error("刷新令牌用户ID不匹配")
            return None
        
        # 生成新的访问令牌
        return self.create_access_token(user_data)
    
    def decode_token_payload(self, token: str) -> Optional[Dict[str, Any]]:
        """
        解码令牌但不验证（用于调试）
        
        Args:
            token: JWT 令牌
            
        Returns:
            解码后的 payload
        """
        try:
            # 注意：不验证签名和过期时间，仅用于调试
            payload = jwt.decode(token, options={"verify_signature": False})
            return payload
        except Exception as e:
            self.logger.error(f"令牌解码失败: {e}")
            return None

# JWT 实战演示
print("=== JWT 认证演示 ===")

# 创建 JWT 管理器
jwt_manager = JWTManager()

# 模拟用户数据
user_data = {
    'user_id': 12345,
    'username': 'john_doe',
    'email': 'john@example.com',
    'roles': ['user', 'premium']
}

print(f"JWT 密钥: {jwt_manager.secret_key[:20]}...")
print(f"算法: {jwt_manager.algorithm}")

# 创建令牌
access_token = jwt_manager.create_access_token(user_data)
refresh_token = jwt_manager.create_refresh_token(user_data['user_id'])

print(f"\n访问令牌: {access_token}")
print(f"刷新令牌: {refresh_token}")

# 验证令牌
print(f"\n验证访问令牌:")
payload = jwt_manager.verify_token(access_token)
if payload:
    print(f"  用户: {payload['username']}")
    print(f"  角色: {payload['roles']}")
    print(f"  过期时间: {datetime.fromtimestamp(payload['exp'])}")
    print(f"  令牌类型: {payload['type']}")

# 解码令牌结构（教学用）
print(f"\n令牌结构解析:")
decoded = jwt_manager.decode_token_payload(access_token)
if decoded:
    print(f"  Header + Payload (未验证): {json.dumps(decoded, indent=2, default=str)}")

# 演示令牌刷新
print(f"\n令牌刷新演示:")
new_access_token = jwt_manager.refresh_access_token(refresh_token, user_data)
if new_access_token:
    print(f"新访问令牌: {new_access_token}")
    print("✅ 令牌刷新成功")
else:
    print("❌ 令牌刷新失败")


## 3. 数据加密与解密

### 加密类型
1. **对称加密**：使用相同密钥加密和解密（AES）
2. **非对称加密**：使用公钥加密，私钥解密（RSA）
3. **哈希函数**：单向函数，不可逆（SHA-256）

### 使用场景
- **敏感数据存储**：用户隐私信息、支付信息
- **数据传输**：HTTPS、API 通信
- **消息认证**：确保数据完整性
- **数字签名**：验证数据来源

### Fernet 对称加密
Fernet 是 cryptography 库提供的对称加密方案，具有以下特点：
- **时间戳验证**：防止重放攻击
- **完整性保护**：防止数据篡改
- **简单易用**：适合大多数应用场景


In [ ]:
class CryptoManager:
    """
    加密管理器 - 处理敏感数据加密
    """
    
    def __init__(self, key: bytes = None):
        """
        初始化加密管理器
        
        Args:
            key: 加密密钥，如果为 None 则生成新密钥
        """
        if key is None:
            key = Fernet.generate_key()
        self.fernet = Fernet(key)
        self.key = key
        self.logger = logging.getLogger(self.__class__.__name__)
    
    def get_key(self) -> bytes:
        """获取加密密钥"""
        return self.key
    
    def encrypt_string(self, plaintext: str) -> str:
        """
        加密字符串
        
        Args:
            plaintext: 明文字符串
            
        Returns:
            加密后的 Base64 字符串
        """
        try:
            encrypted_bytes = self.fernet.encrypt(plaintext.encode('utf-8'))
            encrypted_string = base64.b64encode(encrypted_bytes).decode('utf-8')
            self.logger.info("字符串加密成功")
            return encrypted_string
        except Exception as e:
            self.logger.error(f"字符串加密失败: {e}")
            raise
    
    def decrypt_string(self, encrypted_string: str) -> str:
        """
        解密字符串
        
        Args:
            encrypted_string: 加密的 Base64 字符串
            
        Returns:
            解密后的明文字符串
        """
        try:
            encrypted_bytes = base64.b64decode(encrypted_string.encode('utf-8'))
            decrypted_bytes = self.fernet.decrypt(encrypted_bytes)
            plaintext = decrypted_bytes.decode('utf-8')
            self.logger.info("字符串解密成功")
            return plaintext
        except Exception as e:
            self.logger.error(f"字符串解密失败: {e}")
            raise
    
    def encrypt_dict(self, data: Dict[str, Any]) -> str:
        """
        加密字典数据
        
        Args:
            data: 要加密的字典
            
        Returns:
            加密后的字符串
        """
        json_string = json.dumps(data, ensure_ascii=False)
        return self.encrypt_string(json_string)
    
    def decrypt_dict(self, encrypted_string: str) -> Dict[str, Any]:
        """
        解密字典数据
        
        Args:
            encrypted_string: 加密的字符串
            
        Returns:
            解密后的字典
        """
        json_string = self.decrypt_string(encrypted_string)
        return json.loads(json_string)
    
    def create_secure_token(self, data: Dict[str, Any], expire_hours: int = 24) -> str:
        """
        创建安全令牌（包含过期时间）
        
        Args:
            data: 要包含在令牌中的数据
            expire_hours: 过期小时数
            
        Returns:
            安全令牌
        """
        token_data = {
            'data': data,
            'created_at': datetime.utcnow().isoformat(),
            'expires_at': (datetime.utcnow() + timedelta(hours=expire_hours)).isoformat()
        }
        return self.encrypt_dict(token_data)
    
    def verify_secure_token(self, token: str) -> Optional[Dict[str, Any]]:
        """
        验证安全令牌
        
        Args:
            token: 安全令牌
            
        Returns:
            令牌数据或 None（如果无效或过期）
        """
        try:
            token_data = self.decrypt_dict(token)
            
            # 检查过期时间
            expires_at = datetime.fromisoformat(token_data['expires_at'])
            if datetime.utcnow() > expires_at:
                self.logger.warning("令牌已过期")
                return None
            
            self.logger.info("令牌验证成功")
            return token_data['data']
            
        except Exception as e:
            self.logger.error(f"令牌验证失败: {e}")
            return None

# 加密解密演示
print("=== 数据加密解密演示 ===")

# 创建加密管理器
crypto_manager = CryptoManager()
print(f"生成的加密密钥: {crypto_manager.get_key()}")

# 基本字符串加密
sensitive_data = "用户的银行卡号: 6222 0000 0000 0000"
print(f"\n原始数据: {sensitive_data}")

encrypted = crypto_manager.encrypt_string(sensitive_data)
print(f"加密后: {encrypted}")

decrypted = crypto_manager.decrypt_string(encrypted)
print(f"解密后: {decrypted}")
print(f"解密正确: {sensitive_data == decrypted}")

# 字典数据加密
user_profile = {
    'user_id': 12345,
    'name': '张三',
    'id_card': '110101199001010001',
    'phone': '13800138000',
    'address': '北京市朝阳区某某街道'
}

print(f"\n用户敏感信息: {user_profile}")

encrypted_profile = crypto_manager.encrypt_dict(user_profile)
print(f"加密后的用户信息: {encrypted_profile}")

decrypted_profile = crypto_manager.decrypt_dict(encrypted_profile)
print(f"解密后的用户信息: {decrypted_profile}")

# 安全令牌示例
token_data = {'user_id': 12345, 'action': 'password_reset'}
secure_token = crypto_manager.create_secure_token(token_data, expire_hours=1)
print(f"\n安全令牌: {secure_token}")

verified_data = crypto_manager.verify_secure_token(secure_token)
print(f"验证令牌结果: {verified_data}")

print("✅ 加密解密演示完成")


## 4. 常见 Web 安全威胁与防护

### OWASP Top 10 - 最常见的 Web 安全风险

1. **注入攻击 (Injection)**
   - SQL 注入、命令注入
   - 防护：参数化查询、输入验证

2. **失效的身份认证 (Broken Authentication)**
   - 弱密码、会话管理不当
   - 防护：强密码策略、多因子认证

3. **敏感数据泄露 (Sensitive Data Exposure)**
   - 明文存储、传输不加密
   - 防护：数据加密、HTTPS

4. **XXE (XML External Entity)**
   - XML 外部实体攻击
   - 防护：禁用外部实体、输入验证

5. **失效的访问控制 (Broken Access Control)**
   - 权限绕过、越权访问
   - 防护：最小权限原则、权限验证

6. **安全配置错误 (Security Misconfiguration)**
   - 默认配置、不必要的功能
   - 防护：安全基线、定期审计

7. **跨站脚本 XSS (Cross-Site Scripting)**
   - 注入恶意脚本
   - 防护：输出编码、CSP 策略

8. **不安全的反序列化 (Insecure Deserialization)**
   - 恶意序列化数据
   - 防护：数据验证、签名检查

9. **使用已知漏洞的组件 (Known Vulnerabilities)**
   - 过时的库和框架
   - 防护：依赖扫描、及时更新

10. **不足的日志记录和监控 (Insufficient Logging)**
    - 缺乏安全事件检测
    - 防护：全面日志、实时监控


In [ ]:
class SecurityHelper:
    """
    安全辅助工具类 - 实现常见的安全防护措施
    """
    
    def __init__(self):
        self.logger = logging.getLogger(self.__class__.__name__)
    
    def sanitize_input(self, user_input: str) -> str:
        """
        输入清理 - 防止 XSS 攻击
        
        Args:
            user_input: 用户输入
            
        Returns:
            清理后的输入
        """
        if not user_input:
            return ""
        
        # 简单的 HTML 实体编码
        dangerous_chars = {
            '<': '&lt;',
            '>': '&gt;',
            '"': '&quot;',
            "'": '&#x27;',
            '/': '&#x2F;',
            '&': '&amp;'
        }
        
        sanitized = user_input
        for char, entity in dangerous_chars.items():
            sanitized = sanitized.replace(char, entity)
        
        self.logger.info("输入已清理")
        return sanitized
    
    def generate_csrf_token(self, session_id: str) -> str:
        """
        生成 CSRF 令牌
        
        Args:
            session_id: 会话ID
            
        Returns:
            CSRF 令牌
        """
        # 使用会话ID和时间戳生成令牌
        timestamp = str(int(datetime.utcnow().timestamp()))
        data = f"{session_id}:{timestamp}"
        
        # 使用 HMAC 签名
        secret_key = os.environ.get('CSRF_SECRET', 'default-csrf-secret')
        signature = hmac.new(
            secret_key.encode('utf-8'),
            data.encode('utf-8'),
            hashlib.sha256
        ).hexdigest()
        
        token = f"{data}:{signature}"
        encoded_token = base64.b64encode(token.encode('utf-8')).decode('utf-8')
        
        self.logger.info("CSRF 令牌已生成")
        return encoded_token
    
    def verify_csrf_token(self, token: str, session_id: str, max_age_minutes: int = 60) -> bool:
        """
        验证 CSRF 令牌
        
        Args:
            token: CSRF 令牌
            session_id: 会话ID
            max_age_minutes: 令牌最大有效期（分钟）
            
        Returns:
            验证结果
        """
        try:
            # 解码令牌
            decoded_token = base64.b64decode(token.encode('utf-8')).decode('utf-8')
            parts = decoded_token.split(':')
            
            if len(parts) != 3:
                return False
            
            token_session_id, timestamp, signature = parts
            
            # 验证会话ID
            if token_session_id != session_id:
                self.logger.warning("CSRF 令牌会话ID不匹配")
                return False
            
            # 验证时间戳
            token_time = datetime.fromtimestamp(int(timestamp))
            if datetime.utcnow() - token_time > timedelta(minutes=max_age_minutes):
                self.logger.warning("CSRF 令牌已过期")
                return False
            
            # 验证签名
            data = f"{token_session_id}:{timestamp}"
            secret_key = os.environ.get('CSRF_SECRET', 'default-csrf-secret')
            expected_signature = hmac.new(
                secret_key.encode('utf-8'),
                data.encode('utf-8'),
                hashlib.sha256
            ).hexdigest()
            
            if not hmac.compare_digest(signature, expected_signature):
                self.logger.warning("CSRF 令牌签名无效")
                return False
            
            self.logger.info("CSRF 令牌验证成功")
            return True
            
        except Exception as e:
            self.logger.error(f"CSRF 令牌验证失败: {e}")
            return False
    
    def check_password_breach(self, password: str) -> Dict[str, Any]:
        """
        检查密码是否在已知泄露列表中（模拟实现）
        实际应用中可以集成 HaveIBeenPwned API
        
        Args:
            password: 待检查的密码
            
        Returns:
            检查结果
        """
        # 模拟已知泄露密码列表
        known_breached_passwords = {
            '123456', 'password', '123456789', 'qwerty', 'abc123',
            'password123', 'admin', 'letmein', 'welcome', 'monkey'
        }
        
        is_breached = password.lower() in known_breached_passwords
        
        result = {
            'is_breached': is_breached,
            'risk_level': 'HIGH' if is_breached else 'LOW',
            'message': '此密码已在数据泄露中发现，建议更换' if is_breached else '暂未发现此密码泄露记录'
        }
        
        if is_breached:
            self.logger.warning(f"检测到泄露密码使用")
        
        return result
    
    def generate_api_key(self, user_id: int, permissions: List[str] = None) -> str:
        """
        生成 API 密钥
        
        Args:
            user_id: 用户ID
            permissions: 权限列表
            
        Returns:
            API 密钥
        """
        if permissions is None:
            permissions = ['read']
        
        # 生成随机密钥
        random_key = secrets.token_urlsafe(32)
        
        # 创建密钥数据
        key_data = {
            'user_id': user_id,
            'permissions': permissions,
            'created_at': datetime.utcnow().isoformat(),
            'key_id': secrets.token_hex(8)
        }
        
        # 使用 HMAC 签名
        secret = os.environ.get('API_SECRET', 'default-api-secret')
        key_string = json.dumps(key_data, sort_keys=True)
        signature = hmac.new(
            secret.encode('utf-8'),
            key_string.encode('utf-8'),
            hashlib.sha256
        ).hexdigest()
        
        # 组合最终的 API 密钥
        api_key = f"sk_{key_data['key_id']}_{random_key}_{signature[:16]}"
        
        self.logger.info(f"为用户 {user_id} 生成 API 密钥")
        return api_key

# 安全工具演示
print("=== 安全防护工具演示 ===")
security_helper = SecurityHelper()

# 1. 输入清理演示
print("\n1. XSS 防护 - 输入清理:")
dangerous_input = "<script>alert('XSS Attack!');</script>"
safe_input = security_helper.sanitize_input(dangerous_input)
print(f"危险输入: {dangerous_input}")
print(f"清理后: {safe_input}")

# 2. CSRF 令牌演示
print("\n2. CSRF 防护:")
session_id = "user_session_12345"
csrf_token = security_helper.generate_csrf_token(session_id)
print(f"会话ID: {session_id}")
print(f"CSRF 令牌: {csrf_token}")

# 验证 CSRF 令牌
is_valid = security_helper.verify_csrf_token(csrf_token, session_id)
print(f"令牌验证结果: {is_valid}")

# 错误会话ID验证
is_invalid = security_helper.verify_csrf_token(csrf_token, "wrong_session")
print(f"错误会话验证结果: {is_invalid}")

# 3. 密码泄露检查
print("\n3. 密码泄露检查:")
weak_password = "123456"
strong_password = "MyVerySecurePassword2024!"

weak_result = security_helper.check_password_breach(weak_password)
strong_result = security_helper.check_password_breach(strong_password)

print(f"弱密码检查: {weak_result}")
print(f"强密码检查: {strong_result}")

# 4. API 密钥生成
print("\n4. API 密钥生成:")
api_key = security_helper.generate_api_key(
    user_id=12345, 
    permissions=['read', 'write', 'delete']
)
print(f"生成的 API 密钥: {api_key}")

print("\n✅ 安全防护演示完成")


## 5. 安全最佳实践总结

### 🔐 密码安全
- ✅ 使用 bcrypt 等慢哈希算法
- ✅ 强制密码复杂度要求
- ✅ 检查已知泄露密码
- ❌ 永远不要明文存储密码

### 🎫 身份认证
- ✅ 实施 JWT 双令牌机制（访问+刷新）
- ✅ 设置合理的令牌过期时间
- ✅ 使用强随机密钥
- ✅ 验证令牌完整性和有效期

### 🛡️ 数据保护
- ✅ 敏感数据必须加密存储
- ✅ 传输层使用 HTTPS
- ✅ 实施数据访问控制
- ✅ 定期备份和恢复测试

### 🚫 攻击防护
- ✅ 输入验证和输出编码
- ✅ CSRF 令牌验证
- ✅ SQL 参数化查询
- ✅ 限制文件上传类型和大小

### 📊 监控与审计
- ✅ 记录所有安全相关事件
- ✅ 实时监控异常登录
- ✅ 定期安全扫描
- ✅ 事件响应计划

### 🔧 开发实践
- ✅ 最小权限原则
- ✅ 安全编码规范
- ✅ 依赖库漏洞扫描
- ✅ 代码安全审查

## 下一步学习建议

1. **深入学习**：OAuth 2.0、SAML 等企业级认证
2. **实践项目**：构建完整的用户认证系统
3. **安全工具**：学习使用 OWASP ZAP、Burp Suite
4. **合规要求**：了解 GDPR、PCI DSS 等标准

> 💡 **记住**：安全是一个持续的过程，不是一次性的任务。随着技术发展和威胁演进，我们的安全措施也需要不断更新和完善。
